In [18]:
import os
import sys
import glob
import numpy as np
import pickle
from tqdm import tqdm

In [19]:
import imp
import torch

import utils.data_processor as data_processor
imp.reload(data_processor)
from utils.data_processor import *

import utils.visualization_tools as visualization_tools
imp.reload(visualization_tools)
from utils.visualization_tools import *

import utils.metrics as metrics
imp.reload(metrics)
from utils.metrics import *

# Functions

In [3]:
def binary_3dmaps_to_point_cloud_and_labels(brain, mask, size = 256):
    """ 
    Transforms 3d tensors of brain and hippocampus into pointcloud and labels for it. Both only coordinates and 
    coordinates + intensity modes are suppoted
      Args:
          image_brain: torch tensor of size [size,size,size] with 1 at the positions with brain and 0 otherwise
          image_hypo: torch tensor of size [size,size,size] with 1 at the positions with hippocampus and 0 otherwise
          size: size of the input tensors along each direction, default = 256
          seg: torch tensor of size [size,size,size] with intensities of brain, default None
      Output:
          torch tensor of size [N, 3] is seg is None and [N, 4] otherwise and [N,] tensor with labels
      """
    if type(size) == int:
        grid_x, grid_y, grid_z = torch.meshgrid(torch.tensor(range(size)),\
                                                torch.tensor(range(size)),\
                                                torch.tensor(range(size)))
    else:
        grid_x, grid_y, grid_z = torch.meshgrid(torch.tensor(range(size[0])),\
                                                torch.tensor(range(size[1])),\
                                                torch.tensor(range(size[2])))
        
    new = torch.cat((grid_x.unsqueeze(-1).float(), 
                     grid_y.unsqueeze(-1).float(),
                     grid_z.unsqueeze(-1).float(), 
                     torch.tensor(brain).float().unsqueeze(-1).float()), -1)
    pc_fcd = new[mask==1,:]
    fcd_len = pc_fcd.shape[0]
    idx = np.random.choice(range(fcd_len),fcd_len//100,replace = False)
    pc_fcd = pc_fcd[idx]
    
    
    pc_brain_without_fcd = new[(mask==0)*(brain != 0),:]
    no_fcd_len = pc_brain_without_fcd.shape[0]
    idx = np.random.choice(range(no_fcd_len),no_fcd_len//100,replace = False)
    pc_brain_without_fcd = pc_brain_without_fcd[idx]
    return torch.cat([pc_fcd,pc_brain_without_fcd]),\
np.array([1] * pc_fcd.shape[0] + [0] * pc_brain_without_fcd.shape[0])

In [4]:
def fcd_filename_to_pc_and_labels(file, file_mask, size = 256):
    """ 
    Procceses filename of brain and mask into pointcloud with labels
      Args:
          file: path to brain file
          file_mask: path to mask file
          size: size of the input tensors along each direction is 256, but it can be maxpulled to size. Default = 256
          segfile: file with segmentation path
      Output:
          torch tensor of size [N, 3] is seg is None and [N, 4] otherwise and [N,] tensor with labels
      """
    
    brain = load_nii_to_array(file)
      
    mask = load_nii_to_array(file_mask)
    
    pc,labels = binary_3dmaps_to_point_cloud_and_labels(brain, mask, size = size)
    
    return pc, labels

# Data for experiment creation

In [5]:
SIZE = (316, 374, 309)
UPSAMPLE_RATE = 10
POSTFIX = '_exp1'

In [6]:
pcs, labels = [], []
means_and_stds = []
for file in tqdm(glob.glob('croped_new_dataset/fcd_brains/fcd_*1.nii.gz')):
    peace = file.split('/')[-1]
    try:
        file_mask = glob.glob(f'croped_new_dataset/masks/mask_{peace}*')[0]
    except Exception:
        pass
    pc, label = fcd_filename_to_pc_and_labels(file, file_mask, 
                                  size = SIZE)
    pc = np.array(pc.detach(),dtype = float)
    means_and_stds.append([pc.mean(0), pc.std(0)])
    pcs.append(pc)
    labels.append(label)

100%|██████████| 26/26 [01:58<00:00,  4.55s/it]


In [7]:
sc_labels = [np.array(0)]*len(labels)

In [8]:
for e in tqdm(range(len(pcs))):
    TEST_DATA_INDEXES = [e]
    std = np.array([x[1] for x in (means_and_stds[:e] + means_and_stds[e+1:])]).mean(0)
    mean = np.array([x[0] for x in (means_and_stds[:e] + means_and_stds[e+1:])]).mean(0)
    
    pcs_test,labels_test, sc_labels_test = [(pcs[i]-mean)/std for i in TEST_DATA_INDEXES]*UPSAMPLE_RATE,\
                                            [labels[i] for i in TEST_DATA_INDEXES]*UPSAMPLE_RATE,\ 
                                            [sc_labels[i] for i in TEST_DATA_INDEXES]*UPSAMPLE_RATE
    pcs_train,labels_train, sc_labels_train = [(pcs[i]-mean)/std for i in range(len(pcs)) if i not in TEST_DATA_INDEXES]*UPSAMPLE_RATE,\
                                            [labels[i] for i in range(len(pcs)) if i not in TEST_DATA_INDEXES]*UPSAMPLE_RATE,\
                                            [sc_labels[i] for i in range(len(pcs)) if i not in TEST_DATA_INDEXES]*UPSAMPLE_RATE
    
    data_test = (pcs_test,labels_test, sc_labels_test)
    data_train = (pcs_train,labels_train, sc_labels_train)
    with open(f'data/BrainData/test_data{POSTFIX}_{e}.pkl', 'wb') as f:
        pickle.dump(data_test, f)
    with open(f'data/BrainData/trainval_data{POSTFIX}_{e}.pkl', 'wb') as f:
        pickle.dump(data_train, f)

100%|██████████| 26/26 [00:31<00:00,  1.21s/it]


# Data for pretraining on grey matter

In [32]:
SIZE = (260, 311, 260)
UPSAMPLE_RATE = 1
POSTFIX = '_exp_grey'

In [33]:
def binary_3dmaps_to_point_cloud_and_labels_grey(brain, seg, size = (260, 311, 260)):
    """ 
    Transforms 3d tensors of brain and hippocampus into pointcloud and labels for it. Both only coordinates and 
    coordinates + intensity modes are suppoted
      Args:
          image_brain: torch tensor of size [size,size,size] with 1 at the positions with brain and 0 otherwise
          image_hypo: torch tensor of size [size,size,size] with 1 at the positions with hippocampus and 0 otherwise
          size: size of the input tensors along each direction, default = 256
          seg: torch tensor of size [size,size,size] with intensities of brain, default None
      Output:
          torch tensor of size [N, 3] is seg is None and [N, 4] otherwise and [N,] tensor with labels
      """
    grid_x, grid_y, grid_z = torch.meshgrid(torch.tensor(range(size[0])),\
                                            torch.tensor(range(size[1])),\
                                            torch.tensor(range(size[2])))
    new = torch.cat((grid_x.unsqueeze(-1).float(), 
                     grid_y.unsqueeze(-1).float(),
                     grid_z.unsqueeze(-1).float(), 
                     torch.tensor(brain).float().unsqueeze(-1).float()), -1)
    pc_grey = new[seg>=1000,:]
    grey_len = pc_grey.shape[0]
    idx = np.random.choice(range(grey_len),grey_len//100,replace = False)
    pc_grey = pc_grey[idx]
    
    
    pc_brain_without_grey = new[(seg<1000)*(brain != 0),:]
    no_grey_len = pc_brain_without_grey.shape[0]
    idx = np.random.choice(range(no_grey_len),no_grey_len//100,replace = False)
    pc_brain_without_grey = pc_brain_without_grey[idx]
    return torch.cat([pc_grey,pc_brain_without_grey]),\
np.array([1] * pc_grey.shape[0] + [0] * pc_brain_without_grey.shape[0])

In [34]:
def grey_filename_to_pc_and_labels(intensity, segment, size = (260, 311, 260)):
    """ 
    Procceses filename of brain and mask into pointcloud with labels
      Args:
          file: path to brain file
          file_mask: path to mask file
          size: size of the input tensors along each direction is 256, but it can be maxpulled to size. Default = 256
          segfile: file with segmentation path
      Output:
          torch tensor of size [N, 3] is seg is None and [N, 4] otherwise and [N,] tensor with labels
      """
    
    brain = load_nii_to_array(intensity)
      
    seg = load_nii_to_array(segment)
    
    pc,labels = binary_3dmaps_to_point_cloud_and_labels_grey(brain, seg, size = size)
    
    return pc, labels

In [35]:
pcs, labels = [], []
means_and_stds = []
for file in tqdm(glob.glob('data/HCP_1200/*')):
    segment = file+'/T1w/aparc+aseg.nii.gz'
    intensity = file+'/T1w/T1w_acpc_dc_restore_brain.nii.gz'
    pc, label = grey_filename_to_pc_and_labels(intensity, segment, 
                                  size = SIZE)
    pc = np.array(pc.detach(),dtype = float)
    means_and_stds.append([pc.mean(0), pc.std(0)])
    pcs.append(pc)
    labels.append(label)

100%|██████████| 1113/1113 [29:03<00:00,  1.57s/it]


In [42]:
sc_labels = [np.array(0)]*len(labels)

In [43]:
test_idx = 100

In [46]:
std = np.array([x[1] for x in means_and_stds[test_idx:]]).mean(0)
mean = np.array([x[0] for x in means_and_stds[test_idx:]]).mean(0)
TEST_DATA_INDEXES = list(range(test_idx))
pcs_test,labels_test, sc_labels_test = [(pcs[i]-mean)/std for i in TEST_DATA_INDEXES]*UPSAMPLE_RATE,[labels[i] for i in TEST_DATA_INDEXES]*UPSAMPLE_RATE,[sc_labels[i] for i in TEST_DATA_INDEXES]*UPSAMPLE_RATE
pcs_train,labels_train, sc_labels_train = [(pcs[i]-mean)/std for i in range(len(pcs)) if i not in TEST_DATA_INDEXES]*UPSAMPLE_RATE, [labels[i] for i in range(len(pcs)) if i not in TEST_DATA_INDEXES]*UPSAMPLE_RATE, [sc_labels[i] for i in range(len(pcs)) if i not in TEST_DATA_INDEXES]*UPSAMPLE_RATE

data_test = (pcs_test,labels_test, sc_labels_test)
data_train = (pcs_train,labels_train, sc_labels_train)
with open(f'data/BrainData/test_data{POSTFIX}.pkl', 'wb') as f:
    pickle.dump(data_test, f)
with open(f'data/BrainData/trainval_data{POSTFIX}.pkl', 'wb') as f:
    pickle.dump(data_train, f)